In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy
import sklearn

import os
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt
from sklearn import preprocessing
from sklearn import metrics
import numpy as np
import pandas as pd
from time import time
import pprint
import joblib
from xgboost.sklearn import XGBClassifier 
import numpy as np
from xgboost import XGBClassifier
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings("ignore")

# Classifiers
from catboost import CatBoostClassifier

# Model selection
from sklearn.model_selection import StratifiedKFold

# Metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer

# Skopt functions
from skopt import BayesSearchCV
from skopt.callbacks import DeadlineStopper, VerboseCallback, DeltaXStopper
from skopt.space import Real, Categorical, Integer
from time import time

In [2]:
train_ = pd.read_csv('Train.csv')
test_ = pd.read_csv('Test.csv')
submission_ = pd.read_csv('SampleSubmission.csv')

In [3]:
from sklearn.model_selection import KFold
fold_train = []
kf = KFold(n_splits=5, shuffle=False)
for r, (train_index, test_index) in enumerate(kf.split(train_)):
    test = train_.iloc[test_index]

    X_test = []
    X_test_columns = test.columns
    for v in test.values:
        info = v[:8]
        binary = v[8:]
        index = [k for k, i in enumerate(binary) if i == 1]
        for i in index:
            for k in range(len(binary)):
                if k == i:
                    binary_transformed = list(copy.copy(binary))
                    binary_transformed[i] = 0
                    X_test.append(list(info) + binary_transformed + [8+k])

    X_test = pd.DataFrame(X_test)
    X_test.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
          'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
          '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
          'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'product_num']

    X_test['ID'] = [str(r)+'_'+str(i) for i in range(X_test.shape[0])]

    test = []
    test_columns = X_test.columns

    for v in X_test.values:
        info = v[:8]
        binary = v[8:-1]
        num = v[-1]
        index = [k for k, i in enumerate(binary) if i == 1]
        for k in range(len(binary)):
            if k not in index:
                if 8+k == num:
                    test.append(list(info) + list(binary) + [test_columns[8+k]] + [1])
                else:
                    test.append(list(info) + list(binary) + [test_columns[8+k]] + [0])
    test = pd.DataFrame(test)
    test.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
        'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
        '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
        'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'product_pred', 'target']

    test = test[['ID', 'product_pred', 'target']]
    test['ID X PCODE'] = test['ID'] + ' X ' + test['product_pred']
    del test['ID'], test['product_pred']
    test.columns = ['Label', 'ID X PCODE']

    submit1 = pd.read_csv('0_fold' + str(r) + '.csv')
    submit2 = pd.read_csv('1_fold' + str(r) + '.csv')
    submit3 = pd.read_csv('2_fold' + str(r) + '.csv')

    test = test.merge(submit1, how='left', on='ID X PCODE')
    test = test.merge(submit2, how='left', on='ID X PCODE')
    test = test.merge(submit3, how='left', on='ID X PCODE')

    test.columns = ['target', 'ID X PCODE', 'Label1', 'Label2', 'Label3']

    fold_train.append(test)

fold_train = pd.concat(fold_train)

In [4]:
test = []
test_columns = test_.columns

for v in test_.values:
    info = v[:8]
    binary = v[8:]
    index = [k for k, i in enumerate(binary) if i == 1]
    for k in range(len(binary)):
        if k not in index:
            test.append(list(info) + list(binary) + [test_columns[8+k]])
test = pd.DataFrame(test)
test.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
        'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
        '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
        'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'product_pred',]

test = test[['ID', 'product_pred']]
test['ID X PCODE'] = test['ID'] + ' X ' + test['product_pred']
del test['ID'], test['product_pred']

submit1 = pd.read_csv('0_main.csv')
submit2 = pd.read_csv('1_main.csv')
submit3 = pd.read_csv('2_main.csv')

test = test.merge(submit1, how='left', on='ID X PCODE')
test = test.merge(submit2, how='left', on='ID X PCODE')
test = test.merge(submit3, how='left', on='ID X PCODE')

test.columns = ['ID X PCODE', 'Label1', 'Label2', 'Label3']

In [5]:
test.head()

,ID X PCODE,Label1,Label2,Label3
0,F86J5PC X P5DA,0.000003,0.000915,0.005931
1,F86J5PC X RIBP,0.000655,0.000955,0.003241
2,F86J5PC X 8NN1,0.000007,0.000915,0.005931
3,F86J5PC X 7POT,0.000062,0.000915,0.005931
4,F86J5PC X 66FJ,0.000229,0.000915,0.005931


In [6]:
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

models = []
models.append(LGBMClassifier(random_state=17, num_trees=300, max_depth = 3))
models.append(LGBMClassifier(random_state=17, num_trees=500, max_depth = 5))
models.append(XGBClassifier(learning_rate=0.1675, max_depth=10, gamma=1.976, n_estimators=550))
models.append(XGBClassifier(learning_rate=0.1, max_depth=20, gamma=1.576, n_estimators=750))
models.append(CatBoostClassifier(random_state=17, num_trees=500, max_depth=3))
models.append(CatBoostClassifier(random_state=17, num_trees=1000, max_depth=5))

In [7]:
for i in range(len(models)):
    models[i].fit(fold_train.drop(columns=['target', 'ID X PCODE']), fold_train['target'])

Learning rate set to 0.415293
0:	learn: 0.1173410	total: 750ms	remaining: 6m 14s
1:	learn: 0.0654411	total: 1.39s	remaining: 5m 45s
2:	learn: 0.0395398	total: 1.9s	remaining: 5m 14s
3:	learn: 0.0364164	total: 2.44s	remaining: 5m 2s
4:	learn: 0.0345066	total: 2.93s	remaining: 4m 50s
5:	learn: 0.0343755	total: 3.42s	remaining: 4m 41s
6:	learn: 0.0335156	total: 3.94s	remaining: 4m 37s
7:	learn: 0.0333127	total: 4.4s	remaining: 4m 30s
8:	learn: 0.0331510	total: 5.17s	remaining: 4m 42s
9:	learn: 0.0328362	total: 5.83s	remaining: 4m 45s
10:	learn: 0.0327889	total: 6.5s	remaining: 4m 48s
11:	learn: 0.0326799	total: 6.97s	remaining: 4m 43s
12:	learn: 0.0326233	total: 7.4s	remaining: 4m 37s
13:	learn: 0.0325498	total: 7.76s	remaining: 4m 29s
14:	learn: 0.0325261	total: 8.12s	remaining: 4m 22s
15:	learn: 0.0325104	total: 8.5s	remaining: 4m 17s
16:	learn: 0.0324998	total: 8.92s	remaining: 4m 13s
17:	learn: 0.0324915	total: 9.38s	remaining: 4m 11s
18:	learn: 0.0324548	total: 9.84s	remaining: 4m 9s

In [8]:
predicts = []
for i in range(len(models)):
    predicts.append(models[i].predict_proba(test.drop(columns=['ID X PCODE']))[:, 1])

In [9]:
test['target'] = np.mean(predicts, axis=0)

In [10]:
true_values = []
for v in test_.values:
    binary = v[8:]
    index = [k for k, i in enumerate(binary) if i == 1]
    for k in test_.columns[8:][index]:
        true_values.append(v[0] + ' X ' + k)

In [11]:
test = test[['ID X PCODE', 'target']]
test.columns = ['ID X PCODE', 'Label']

In [12]:
for v in true_values:
    test = test.append({'ID X PCODE':v,'Label':1.0}, ignore_index=True)

In [13]:
test.reset_index(drop=True, inplace=True)
test.to_csv('submission27.csv', index=False)

OK